In [ ]:
import cv2
import numpy as np
import easyocr
from matplotlib import pyplot as plt
import os

# ==========================================
# 1. LOGIKA CAMSCANNER (PERSPECTIVE TRANSFORM)
# ==========================================
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return warped

def auto_scan_document(image):
    orig = image.copy()
    gray = cv2.GaussianBlur(image, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)

    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

    screenCnt = None
    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            screenCnt = approx
            break

    if screenCnt is not None:
        print("[INFO] Strategi 1: CamScanner Mode (4 Sudut Ditemukan)")
        pts = screenCnt.reshape(4, 2)
        warped = four_point_transform(orig, pts)
        return warped, True
    else:
        print("[INFO] Strategi 1 Gagal (Sudut tidak jelas). Lanjut ke Strategi 2.")
        return image, False

# ==========================================
# 2. LOGIKA SIMPLE DESKEW (ROTASI BIASA)
# ==========================================
def deskew_simple(image, delta=1.0):
    gray = image.copy()
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    contours, _ = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    if len(contours) == 0:
        return image
    
    largest_contour = contours[0]
    minAreaRect = cv2.minAreaRect(largest_contour)
    angle = minAreaRect[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
        
    if abs(angle) < delta or abs(angle) > 45:
        print(f"[INFO] Strategi 2: Sudut {angle:.2f} derajat (Abaikan)")
        return image

    print(f"[INFO] Strategi 2: Rotasi Biasa ({angle:.2f} derajat)")
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return rotated

# ==========================================
# 3. LOGIKA ORIENTASI (PORTRAIT ENFORCER)
# ==========================================
def ensure_portrait(image):
    (h, w) = image.shape[:2]
    if w > h:
        print("[INFO] Orientasi Landscape terdeteksi. Memutar 90 derajat jadi Portrait...")
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    return image

# ==========================================
# 4. TUNING LAB (ORCHESTRATOR)
# ==========================================
def tuning_lab(image, blur_kernel=21, denoise_h=10, upscale=False):
    if image is None: return None
    
    # Geometri
    processed_img, success = auto_scan_document(image)
    if not success:
        processed_img = deskew_simple(processed_img)
    processed_img = ensure_portrait(processed_img)

    if upscale:
        processed_img = cv2.resize(processed_img, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)

    # Visualisasi (Binary) vs Data (Grayscale)
    visual_binary = cv2.adaptiveThreshold(
        processed_img, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 15, 10
    )
    
    # Denoising untuk EasyOCR
    ocr_ready_img = cv2.fastNlMeansDenoising(processed_img, None, h=denoise_h, templateWindowSize=7, searchWindowSize=21)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1); plt.imshow(image, cmap='gray'); plt.title("Original"); plt.axis('off')
    plt.subplot(1, 2, 2); plt.imshow(visual_binary, cmap='gray'); plt.title("Visual Check"); plt.axis('off')
    plt.show()
    
    return ocr_ready_img

# ==========================================
# [BARU] 5. LINE GROUPING LOGIC (PENGGABUNG BARIS)
# ==========================================
def group_ocr_results(results, y_threshold=15):
    """
    Menggabungkan kata-kata yang terpisah menjadi satu baris kalimat
    berdasarkan kedekatan koordinat Y (Vertikal).
    """
    # results format dari EasyOCR detail=1: ([[x,y]...], 'text', conf)
    
    # 1. Urutkan semua kata berdasarkan posisi Y (atas ke bawah)
    # Kita pakai rata-rata Y dari kotak bounding box
    def get_avg_y(box):
        return sum([p[1] for p in box]) / 4

    sorted_results = sorted(results, key=lambda x: get_avg_y(x[0]))
    
    lines = []
    current_line = []
    
    for i, (bbox, text, conf) in enumerate(sorted_results):
        # Hitung posisi Y rata-rata kata saat ini
        current_y = get_avg_y(bbox)
        
        if not current_line:
            current_line.append((bbox, text))
            continue
            
        # Ambil posisi Y rata-rata dari kata terakhir di baris saat ini
        last_bbox = current_line[-1][0]
        last_y = get_avg_y(last_bbox)
        
        # Jika jarak vertikalnya kecil (kurang dari threshold), anggap satu baris
        if abs(current_y - last_y) < y_threshold:
            current_line.append((bbox, text))
        else:
            # Jika jaraknya jauh, berarti sudah ganti baris
            lines.append(current_line)
            current_line = [(bbox, text)]
            
    # Jangan lupa masukkan baris terakhir
    if current_line:
        lines.append(current_line)
        
    # 2. Gabungkan Teks dalam setiap baris
    final_text = []
    for line in lines:
        # Urutkan kata dalam satu baris berdasarkan X (kiri ke kanan)
        line.sort(key=lambda x: x[0][0][0]) 
        
        # Gabungkan jadi satu string
        combined_text = " ".join([item[1] for item in line])
        final_text.append(combined_text)
        
    return "\n".join(final_text)

# ==========================================
# 6. MAIN EXECUTION
# ==========================================
img_path = "../../dataset/test/img/abel.jpg" 

if os.path.exists(img_path):
    original_img = cv2.imread(img_path)
    gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    
    print("\n=== MULAI SMART SCANNING ===")
    result_image = tuning_lab(gray_img, blur_kernel=21, denoise_h=10, upscale=True)

    if result_image is not None:
        print("\n=== HASIL BACAAN OCR ===")
        reader = easyocr.Reader(['en', 'id'], gpu=False)
        
        # PENTING: Ubah detail=0 jadi detail=1 agar dapat koordinat
        raw_results = reader.readtext(result_image, detail=1) 
        
        # Panggil fungsi penggabung baris
        final_clean_text = group_ocr_results(raw_results, y_threshold=20) # Threshold bisa diatur (15-25 pixel)
        
        print(final_clean_text)
else:
    print(f"❌ Gambar tidak ditemukan di path: {img_path}")